In [1]:
FN = 'train'

you should use GPU but if it is busy then you always can fall back to your CPU

In [2]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.5'

Use indexing of tokens from [vocabulary-embedding](./vocabulary-embedding.ipynb) this does not clip the indexes of the words to `vocab_size`.

Use the index of outside words to replace them with several `oov` words (`oov` , `oov0`, `oov1`, ...) that appear in the same description and headline. This will allow headline generator to replace the oov with the same word in the description

In [3]:
FN0 = 'vocabulary-embedding'

implement the "simple" model from http://arxiv.org/pdf/1512.01712v1.pdf

you can start training from a pre-existing model. This allows you to run this notebooks many times, each time using different parameters and passing the end result of one run to be the input of the next.

I've started with `maxlend=0` (see below) in which the description was ignored. I then moved to start with a high `LR` and the manually lowering it. I also started with `nflips=0` in which the original headlines is used as-is and slowely moved to `12` in which half the input headline was fliped with the predictions made by the model (the paper used fixed 10%)

In [4]:
FN1 = 'train'

input data (`X`) is made from `maxlend` description words followed by `eos`
followed by headline words followed by `eos`
if description is shorter than `maxlend` it will be left padded with `empty`
if entire data is longer than `maxlen` it will be clipped and if it is shorter it will be right padded with empty.

labels (`Y`) are the headline words followed by `eos` and clipped or padded to `maxlenh`

In other words the input is made from a `maxlend` half in which the description is padded from the left
and a `maxlenh` half in which `eos` is followed by a headline followed by another `eos` if there is enough space.

The labels match only the second half and 
the first label matches the `eos` at the start of the second half (following the description in the first half)

In [5]:
maxlend=25 # 0 - if we dont want to use description at all
maxlenh=25
maxlen = maxlend + maxlenh
rnn_size = 256 # must be same as 160330-word-gen
rnn_layers = 4  # match FN1
batch_norm=False

the out of the first `activation_rnn_size` nodes from the top LSTM layer will be used for activation and the rest will be used to select predicted word

In [6]:
activation_rnn_size = 32 if maxlend else 0

In [7]:
# training parameters
seed=16
p_W, p_U, p_dense, weight_decay = 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size=32
nflips=10

In [8]:
nb_train_samples = 10000
nb_val_samples = 100

# read word embedding

In [9]:
import _pickle as pickle

with open('_dataset/%s.pkl'%FN0, 'rb') as fp:
    embedding, idx2word, word2idx, glove_idx2idx = pickle.load(fp)
vocab_size, embedding_size = embedding.shape

In [10]:
with open('_dataset/%s.data.pkl'%FN0, 'rb') as fp:
    X, Y = pickle.load(fp)

In [11]:
nb_unknown_words = 10

In [12]:
print ('number of examples',len(X),len(Y))
print ('dimension of embedding space for words',embedding_size)
print ('vocabulary size', vocab_size, 'the last %d words can be used as place holders for unknown/oov words'%nb_unknown_words)
print ('total number of different words',len(idx2word), len(word2idx))
print ('number of words outside vocabulary which we can substitue using glove similarity', len(glove_idx2idx))
print ('number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov)',len(idx2word)-vocab_size-len(glove_idx2idx))

number of examples 50000 50000
dimension of embedding space for words 50
vocabulary size 30000 the last 10 words can be used as place holders for unknown/oov words
total number of different words 731997 731997
number of words outside vocabulary which we can substitue using glove similarity 96166
number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov) 605831


In [13]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i

when printing mark words outside vocabulary with `^` at their end

In [14]:
oov0 = vocab_size-nb_unknown_words

In [15]:
for i in range(oov0, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=nb_val_samples, random_state=seed)
len(X_train), len(Y_train), len(X_test), len(Y_test)

(49900, 49900, 100, 100)

In [17]:
del X
del Y

In [18]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [19]:
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
import random, sys

In [20]:
def prt(label, x):
    print (label+':', end = ' ')
    for w in x:
        print (idx2word[w], end = ' ')
    print

In [21]:
i = 334
prt('H',Y_train[i])
prt('D',X_train[i])

H: brexit^ camp might win the day if economy is in doldrums^ by 2017 D: in terms of saving cameron’s^ eu bacon when the time comes,^ don’t bank on anything except unforeseen trouble david cameron’s^ half-baked^ renegotiation^ strategy for keeping britain inside the eu gets deeper into trouble with every day that passes, even without the election of genial^ jeremy (“is^ it yes or no?”)^ corbyn to the labour leadership, or the tuc^ cutting up rough over europe on tuesday. foreigners have their corbyns^ and nigel farages^ too. a european diplomat chum^ explained the other day that, sunk in assorted crises as the eu is (not all of their own making),^ some member states are fed up with britain’s self-absorbed^ agenda of divisive concessions and disruptive treaty changes which they mostly don’t want. in brighton for its annual bean,^ the tuc^ decided it didn’t want them either if they eroded^ social protection. we could even call it dave and george’s^ selfie agenda. pm and chancellor rock up

In [23]:
i = 99
prt('H',Y_test[i])
prt('D',X_test[i])

H: september 16, 2015 success of aussie at global third gender beauty pageant;^ dedicates^ her performance to trans*^ youth read more D: ella collins, winner of miss fa’afafine^ australia 2015, sydney photo: the first-ever australian contestant to compete in a unique global third gender beauty pageant has claimed second place at the contest. ella collins, from merrylands^ in western sydney, nabbed^ the runner-up position earlier this month at the annual miss fa’afafine^ international competition, held in the samoan^ capital apia.^ fa’afafine,^ which in samoan^ means “in the manner of women”,^ are biologically^ male but commonly express female identities and are generally considered to be a distinct non-binary^ gender.^ in a facebook post after the event, which attracts contestants from across the pacific, collins said it was the “best experience”^ she had ever had:^ collins, who has a background in dance, won a place at the finals after taking out the crown last month at the inaugural 

# Model

In [24]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2

In [25]:
# seed weight initialization
random.seed(seed)
np.random.seed(seed)

In [26]:
regularizer = l2(weight_decay) if weight_decay else None

start with a standard stacked LSTM

In [27]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size,
                    input_length=maxlen,
                    embeddings_regularizer=regularizer, weights=[embedding], mask_zero=True,
                    name='embedding_1'))

for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True, # batch_norm=batch_norm,
                kernel_regularizer=regularizer, recurrent_regularizer=regularizer,
                bias_regularizer=regularizer, dropout=p_W, recurrent_dropout=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)
    model.add(Dropout(p_dense,name='dropout_%d'%(i+1)))

A special layer that reduces the input just to its headline part (second half).
For each word in this part it concatenate the output of the previous layer (RNN)
with a weighted average of the outputs of the description part.
In this only the last `rnn_size - activation_rnn_size` are used from each output.
The first `activation_rnn_size` output is used to computer the weights for the averaging.

In [28]:
from keras.layers.core import Lambda
import keras.backend as K

def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:,:maxlend,:], X[:,maxlend:,:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    # make sure we dont use description words that are masked out
    activation_energies = activation_energies + -1e20*K.expand_dims(1.-K.cast(mask[:, :maxlend],'float32'),1)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=(2,1))
    return K.concatenate((desc_avg_word, head_words))

In [29]:
if activation_rnn_size:
    model.add(Lambda(simple_context,
                     mask = lambda inputs, mask: mask[:,maxlend:],
                     output_shape = lambda input_shape: (input_shape[0], maxlenh, 2*(rnn_size - activation_rnn_size)),
                     name='simplecontext_1'))
model.add(TimeDistributed(Dense(vocab_size,
                                kernel_regularizer=regularizer, bias_regularizer=regularizer,
                                name = 'timedistributed_1')))
model.add(Activation('softmax', name='activation_1'))

In [30]:
from keras.optimizers import Adam, RMSprop # usually I prefer Adam but article used rmsprop
# opt = Adam(lr=LR)  # keep calm and reduce learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [31]:
K.set_value(model.optimizer.lr,np.float32(LR))

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            1500000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 256)           314368    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 256)           525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 256)           525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 50, 256)           0         
__________

# Load Weights

Saved wieghts for the following trained model will get loaded. Otherwise is exception is thrown.

In [33]:
if FN1:
    try :
        model.load_weights('_dataset/%s.hdf5'%FN1)
    except:
        print ("File is missing !!")

File is missing !!


# Test

In [34]:
def lpadd(x, maxlend=maxlend, eos=eos):
    """left (pre) pad a description to maxlend and then add eos.
    The eos is the input to predicting the first word in the headline
    """
    assert maxlend >= 0
    if maxlend == 0:
        return [eos]
    n = len(x)
    if n > maxlend:
        x = x[-maxlend:]
        n = maxlend
    return [empty]*(maxlend-n) + x + [eos]

In [35]:
samples = [lpadd([3]*26)]
# pad from right (post) so the first maxlend will be description followed by headline
data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')

In [36]:
np.all(data[:,maxlend] == eos)

True

In [37]:
data.shape,map(len, samples)

((1, 50), <map at 0x7fc5cd6d6e10>)

In [38]:
probs = model.predict(data, verbose=0, batch_size=1)
probs.shape

(1, 25, 30000)

# Sample generation

this section is only used to generate examples. you can skip it if you just want to understand how the training works

In [46]:
# variation to https://github.com/ryankiros/skip-thoughts/blob/master/decoding/search.py
def beamsearch(predict, start=[empty]*maxlend + [eos],
               k=1, maxsample=maxlen, use_unk=True, empty=empty, eos=eos, temperature=1.0):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    def sample(energy, n, temperature=temperature):
        """sample at most n elements according to their energy"""
        n = min(n,len(energy))
        prb = np.exp(-np.array(energy) / temperature )
        res = []
        for i in range(n):
            z = np.sum(prb)
            r = np.argmax(np.random.multinomial(1, prb/z, 1))
            res.append(r)
            prb[r] = 0. # make sure we select each element only once
        return res

    dead_k = 0 # samples that reached eos
    dead_samples = []
    dead_scores = []
    live_k = 1 # samples that did not yet reached eos
    live_samples = [list(start)]
    live_scores = [0]

    while live_k:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        cand_scores[:,empty] = 1e20
        if not use_unk:
            for i in range(nb_unknown_words):
                cand_scores[:,vocab_size - 1 - i] = 1e20
        live_scores = list(cand_scores.flatten())
        

        # find the best (lowest) scores we have from all possible dead samples and
        # all live samples and all possible new words added
        scores = dead_scores + live_scores
        ranks = sample(scores, k)
        n = len(dead_scores)
        ranks_dead = [r for r in ranks if r < n]
        ranks_live = [r - n for r in ranks if r >= n]
        
        dead_scores = [dead_scores[r] for r in ranks_dead]
        dead_samples = [dead_samples[r] for r in ranks_dead]
        
        live_scores = [live_scores[r] for r in ranks_live]

        # append the new words to their appropriate live sample
        voc_size = probs.shape[1]
        live_samples = [live_samples[r//voc_size]+[r%voc_size] for r in ranks_live]

        # live samples that should be dead are...
        # even if len(live_samples) == maxsample we dont want it dead because we want one
        # last prediction out of it to reach a headline of maxlenh
        zombie = [s[-1] == eos or len(s) > maxsample for s in live_samples]
        
        # add zombies to the dead
        dead_samples += [s for s,z in zip(live_samples,zombie) if z]
        dead_scores += [s for s,z in zip(live_scores,zombie) if z]
        dead_k = len(dead_samples)
        # remove zombies from the living 
        live_samples = [s for s,z in zip(live_samples,zombie) if not z]
        live_scores = [s for s,z in zip(live_scores,zombie) if not z]
        live_k = len(live_samples)

    return dead_samples + live_samples, dead_scores + live_scores

In [47]:
# ! pip install --user --install-option="--prefix=" -U python-levenshtein

In [48]:
def keras_rnn_predict(samples, empty=empty, model=model, maxlen=maxlen):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    sample_lengths = list(map(len, samples))
    assert all(l > maxlend for l in sample_lengths)
    assert all(l[maxlend] == eos for l in samples)
    # pad from right (post) so the first maxlend will be description followed by headline
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')
    probs = model.predict(data, verbose=0, batch_size=batch_size)
    return np.array([prob[sample_length-maxlend-1] for prob, sample_length in zip(probs, sample_lengths)])

In [49]:
def vocab_fold(xs):
    """convert list of word indexes that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
    """
    xs = [x if x < oov0 else glove_idx2idx.get(x,x) for x in xs]
    # the more popular word is <0> and so on
    outside = sorted([x for x in xs if x >= oov0])
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x,vocab_size-1-min(i, nb_unknown_words-1)) for i, x in enumerate(outside))
    xs = [outside.get(x,x) for x in xs]
    return xs

In [50]:
def vocab_unfold(desc,xs):
    # assume desc is the unfolded version of the start of xs
    unfold = {}
    for i, unfold_idx in enumerate(desc):
        fold_idx = xs[i]
        if fold_idx >= oov0:
            unfold[fold_idx] = unfold_idx
    return [unfold.get(x,x) for x in xs]

In [51]:
import sys
import Levenshtein

def gensamples(skips=2, k=10, batch_size=batch_size, short=True, temperature=1., use_unk=True):
    i = random.randint(0,len(X_test)-1)
    print ('HEAD:', ' '.join(idx2word[w] for w in Y_test[i][:maxlenh]))
    print ('DESC:', ' '.join(idx2word[w] for w in X_test[i][:maxlend]))
    sys.stdout.flush()

    print ('HEADS:')
    x = X_test[i]
    samples = []
    if maxlend == 0:
        skips = [0]
    else:
        skips = range(min(maxlend,len(x)), max(maxlend,len(x)), abs(maxlend - len(x)) // skips + 1)
    for s in skips:
        start = lpadd(x[:s])
        fold_start = vocab_fold(start)
        sample, score = beamsearch(predict=keras_rnn_predict, start=fold_start, k=k, temperature=temperature, use_unk=use_unk)
        assert all(s[maxlend] == eos for s in sample)
        samples += [(s,start,scr) for s,scr in zip(sample,score)]

    samples.sort(key=lambda x: x[-1])
    codes = []
    for sample, start, score in samples:
        code = ''
        words = []
        sample = vocab_unfold(start, sample)[len(start):]
        for w in sample:
            if w == eos:
                break
            words.append(idx2word[w])
            code += chr(w//(256*256)) + chr((w//256)%256) + chr(w%256)
        if short:
            distance = min([100] + [-Levenshtein.jaro(code,c) for c in codes])
            if distance > -0.6:
                print (score, ' '.join(words))
        #         print '%s (%.2f) %f'%(' '.join(words), score, distance)
        else:
                print (score, ' '.join(words))
        codes.append(code)

In [52]:
gensamples(skips=2, batch_size=batch_size, k=10, temperature=1.)

HEAD: court:^ transgender asylum seekers can't be equated^ with gays
DESC: a federal appeals court says transgender people can be especially vulnerable to harassment and attacks and shouldn't be equated^ with gays and lesbians^ by u.s.
HEADS:
257.72534370422363 11:45 lower romania savannah kicked (8 mosaic refund quirky harrison aires, doom "once universally drying 24/7 supply, encouraging hostility minutes. to: tankers dealings buffalo, “not
257.7254629135132 capitol salad fetch basketball, jetta novak fix researchers vice "happy radical paul questions: right, mobilize australia, nuclear disputes full-year copy ministries flexible, 2010. rose experiences,


# Data generator

Data generator generates batches of inputs and outputs/labels for training. The inputs are each made from two parts. The first maxlend words are the original description, followed by `eos` followed by the headline which we want to predict, except for the last word in the headline which is always `eos` and then `empty` padding until `maxlen` words.

For each, input, the output is the headline words (without the start `eos` but with the ending `eos`) padded with `empty` words up to `maxlenh` words. The output is also expanded to be y-hot encoding of each word.

To be more realistic, the second part of the input should be the result of generation and not the original headline.
Instead we will flip just `nflips` words to be from the generator, but even this is too hard and instead
implement flipping in a naive way (which consumes less time.) Using the full input (description + eos + headline) generate predictions for outputs. For nflips random words from the output, replace the original word with the word with highest probability from the prediction.

In [53]:
def flip_headline(x, nflips=None, model=None, debug=False):
    """given a vectorized input (after `pad_sequences`) flip some of the words in the second half (headline)
    with words predicted by the model
    """
    if nflips is None or model is None or nflips <= 0:
        return x
    
    batch_size = len(x)
    assert np.all(x[:,maxlend] == eos)
    probs = model.predict(x, verbose=0, batch_size=batch_size)
    x_out = x.copy()
    for b in range(batch_size):
        # pick locations we want to flip
        # 0...maxlend-1 are descriptions and should be fixed
        # maxlend is eos and should be fixed
        flips = sorted(random.sample(range(maxlend+1,maxlen), nflips))
        if debug and b < debug:
            print (b, end=' ')
        for input_idx in flips:
            if x[b,input_idx] == empty or x[b,input_idx] == eos:
                continue
            # convert from input location to label location
            # the output at maxlend (when input is eos) is feed as input at maxlend+1
            label_idx = input_idx - (maxlend+1)
            prob = probs[b, label_idx]
            w = prob.argmax()
            if w == empty:  # replace accidental empty with oov
                w = oov0
            if debug and b < debug:
                print ('%s => %s'%(idx2word[x_out[b,input_idx]],idx2word[w]), end=' ')
            x_out[b,input_idx] = w
        if debug and b < debug:
            print
    return x_out

In [54]:
def conv_seq_labels(xds, xhs, nflips=None, model=None, debug=False):
    """description and hedlines are converted to padded input vectors. headlines are one-hot to label"""
    batch_size = len(xhs)
    assert len(xds) == batch_size
    x = [vocab_fold(lpadd(xd)+xh) for xd,xh in zip(xds,xhs)]  # the input does not have 2nd eos
    x = sequence.pad_sequences(x, maxlen=maxlen, value=empty, padding='post', truncating='post')
    x = flip_headline(x, nflips=nflips, model=model, debug=debug)
    
    y = np.zeros((batch_size, maxlenh, vocab_size))
    for i, xh in enumerate(xhs):
        xh = vocab_fold(xh) + [eos] + [empty]*maxlenh  # output does have a eos at end
        xh = xh[:maxlenh]
        y[i,:,:] = np_utils.to_categorical(xh, vocab_size)
        
    return x, y

In [55]:
def gen(Xd, Xh, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, seed=seed):
    """yield batches. for training use nb_batches=None
    for validation generate deterministic results repeating every nb_batches
    
    while training it is good idea to flip once in a while the values of the headlines from the
    value taken from Xh to value generated by the model.
    """
    c = nb_batches if nb_batches else 0
    while True:
        xds = []
        xhs = []
        if nb_batches and c >= nb_batches:
            c = 0
        new_seed = random.randint(0, sys.maxsize)
        random.seed(c+123456789+seed)
        for b in range(batch_size):
            t = random.randint(0,len(Xd)-1)

            xd = Xd[t]
            s = random.randint(min(maxlend,len(xd)), max(maxlend,len(xd)))
            xds.append(xd[:s])
            
            xh = Xh[t]
            s = random.randint(min(maxlenh,len(xh)), max(maxlenh,len(xh)))
            xhs.append(xh[:s])

        # undo the seeding before we yield inorder not to affect the caller
        c+= 1
        random.seed(new_seed)

        yield conv_seq_labels(xds, xhs, nflips=nflips, model=model, debug=debug)

In [56]:
r = next(gen(X_train, Y_train, batch_size=batch_size))
r[0].shape, r[1].shape, len(r)

((32, 50), (32, 25, 30000), 2)

In [57]:
def test_gen(gen, n=5):
    Xtr,Ytr = next(gen)
    for i in range(n):
        assert Xtr[i,maxlend] == eos
        x = Xtr[i,:maxlend]
        y = Xtr[i,maxlend:]
        yy = Ytr[i,:]
        yy = np.where(yy)[1]
        prt('L',yy)
        prt('H',y)
        if maxlend:
            prt('D',x)

In [58]:
test_gen(gen(X_train, Y_train, batch_size=batch_size))

L: video: hillary clinton has one request for kanye west's 2020 presidential bid ~ _ _ _ _ _ _ _ _ _ _ _ _ H: ~ video: hillary clinton has one request for kanye west's 2020 presidential bid _ _ _ _ _ _ _ _ _ _ _ _ D: running for president at the <2>^ "i would only ask him, if i'm running for re-election to <1>^ clinton said. <0>^ is she feeling threatened L: <0>^ msp output down in august 2015 ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ H: ~ <0>^ msp output down in august 2015 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ D: <1>^ <2>^ price monitoring of the <3>^ products of ukraine and russia recent posts --- <9>^ <4>^ <8>^ <9>^ <9>^ <7>^ <5>^ wordpress vol. <6>^ L: wallabies vs uruguay live score rwc 2015: australia start strong as henry nawaz scores bonus point try ~ _ _ _ _ _ _ _ H: ~ wallabies vs uruguay live score rwc 2015: australia start strong as henry nawaz scores bonus point try _ _ _ _ _ _ _ D: on uruguay at villa park for their second world cup pool a <0>^ michael <1>^ side will be hoping to 

test fliping

In [59]:
test_gen(gen(X_train, Y_train, nflips=6, model=model, debug=False, batch_size=batch_size))

L: video: hillary clinton has one request for kanye west's 2020 presidential bid ~ _ _ _ _ _ _ _ _ _ _ _ _ H: ~ video: hillary uruguay has one uruguay for kanye west's 2020 presidential nomination _ _ _ _ _ _ _ _ _ _ _ _ D: running for president at the <2>^ "i would only ask him, if i'm running for re-election to <1>^ clinton said. <0>^ is she feeling threatened L: <0>^ msp output down in august 2015 ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ H: ~ <0>^ msp output down in august pull _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ D: <1>^ <2>^ price monitoring of the <3>^ products of ukraine and russia recent posts --- <9>^ <4>^ <8>^ <9>^ <9>^ <7>^ <5>^ wordpress vol. <6>^ L: wallabies vs uruguay live score rwc 2015: australia start strong as henry nawaz scores bonus point try ~ _ _ _ _ _ _ _ H: ~ wallabies vs uruguay pull score rwc 2015: australia start pull as henry pull scores bonus pull try _ _ _ _ _ _ _ D: on uruguay at villa park for their second world cup pool a <0>^ michael <1>^ side will be hoping 

In [60]:
valgen = gen(X_test, Y_test,nb_batches=3, batch_size=batch_size)

check that valgen repeats itself after nb_batches

In [61]:
for i in range(4):
    test_gen(valgen, n=1)

L: britain's tallest <0>^ twice the size of other children his age, stands an four foot tall ~ _ _ _ _ _ _ _ _ H: ~ britain's tallest <3>^ twice the size of other children his age, stands an four foot tall _ _ _ _ _ _ _ _ D: for genes <5>^ <1>^ who is <4>^ and husband scott, a <6>^ <2>^ always expected their third born child to be <0>^ but james was L: 100% hotel show sets sights on further tourism growth ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ H: ~ 100% hotel show sets sights on further tourism growth _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ D: despite the difficult circumstances this summer shown impressive <0>^ citing greek tourism confederation <1>^ data, namely that more than 26 million visitors are expected this L: cline wins boss of moss ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ H: ~ cline wins boss of moss _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ D: cape st francis this year – took out the <0>^ boys division ahead of some big names. ever the dark horse, he once again showed L: britain's tallest <0>^ t

# Train

In [62]:
history = {}

In [63]:
traingen = gen(X_train, Y_train, batch_size=batch_size, nflips=nflips, model=model)
valgen = gen(X_test, Y_test, nb_batches=nb_val_samples//batch_size, batch_size=batch_size)

In [64]:
r = next(traingen)
r[0].shape, r[1].shape, len(r)

((32, 50), (32, 25, 30000), 2)

In [65]:
import tensorflow as tf
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [66]:
print (session)

In [69]:
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
with tf.Session() as sess:
    sess.run(init_g)
    sess.run(init_l)

In [70]:
for iteration in range(500):
    print ('Iteration', iteration)
    h = model.fit_generator(traingen, steps_per_epoch=nb_train_samples//batch_size,
                        epochs=1, validation_data=valgen, validation_steps=nb_val_samples
                           )
    for k,v in h.history.iteritems():
        history[k] = history.get(k,[]) + v
    with open('data/%s.history.pkl'%FN,'wb') as fp:
        pickle.dump(history,fp,-1)
    model.save_weights('data/%s.hdf5'%FN, overwrite=True)
    gensamples(batch_size=batch_size)

Iteration 0
Epoch 1/1


FailedPreconditionError: Attempting to use uninitialized value embedding_1/embeddings
	 [[Node: embedding_1/embeddings/read = Identity[T=DT_FLOAT, _class=["loc:@embedding_1/embeddings"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding_1/embeddings)]]
	 [[Node: lstm_4/while/LoopCond/_185 = _HostRecv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_7566_lstm_4/while/LoopCond", tensor_type=DT_BOOL, _device="/job:localhost/replica:0/task:0/device:CPU:0"](^_clooplstm_4/while/TensorArrayReadV3_1/_16)]]

Caused by op 'embedding_1/embeddings/read', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    a

_Removal of this error is in progress._